## [--- Day 11: Reactor ---](https://adventofcode.com/2025/day/11)

In [ ]:
import re
from dataclasses import dataclass, field
from collections import deque


@dataclass
class Node:
    name: str
    parents: set["Node"] = field(default_factory=set)
    children: set["Node"] = field(default_factory=set)

    def __hash__(self) -> int:
        return hash(self.name)


def solve() -> None:
    with open("..\\data\\11.txt") as file:
        pattern: re.Pattern = re.compile(r"\w{3}")
        nodes: dict[str, Node] = {}
        child_map: dict[str, list[str]] = {}

        for line in file.readlines():
            matches = pattern.findall(line)
            node = Node(name=matches[0])
            child_map[node.name] = matches[1:]
            nodes[node.name] = node

        nodes["out"] = Node(name="out")

    for name, children in child_map.items():
        for child in children:
            nodes[name].children.add(nodes[child])
            nodes[child].parents.add(nodes[name])

    # Assign graph depth, enabling iteration in topological order
    in_degrees: dict[Node, int] = {node: 0 for node in nodes.values()}
    for node in nodes.values():
        for child in node.children:
            in_degrees[child] += 1

    # Kahn's algorithm. Starting from all degree 0 nodes, but only "you" has a path to propagate
    path_counts: dict[Node, int] = {node: 0 for node in nodes.values()}
    path_counts[nodes["you"]] = 1

    queue: deque[Node] = deque(
        [node for node in nodes.values() if in_degrees[node] == 0]
    )
    while queue:
        current_node = queue.popleft()
        current_count = path_counts[current_node]
        for child in current_node.children:
            path_counts[child] += current_count
            in_degrees[child] -= 1
            if in_degrees[child] == 0:
                queue.append(child)

    print(path_counts[nodes["out"]])


solve()

791
